In [ ]:
import numpy as np
import random

In [ ]:
population_size=1200
dimension_Etask=50
dimension_Ctask1=35
dimension_Ctask2=25
dimension_Ctask3=50
dimension_max=50
shift_Etask=np.concatenate((np.full(25, 10), np.zeros(25)))
shift_Ctask1=np.full(35, 10)
shift_Ctask2=np.full(25, 0)
shift_Ctask3=np.random.uniform(0, 10, size=50)

In [ ]:
frequence_translate=0
threshold=0
maxgen=1200
cp=np.full(dimension_max, 0.5)
sf=1.25


In [ ]:
def rosenbrock(x):
    f = 0

    for i in range(len(x) - 1):
        f += 100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return f
def rastrigin(x):

    n = len(x)
    A = 10
    f = A * n
    for i in range(n):
        f += x[i]**2 - A * np.cos(2 * np.pi * x[i])
    return f
def E_Task(x,o):
  x=x[:dimension_Etask]
  return rosenbrock(x[25:]-o[25:])+rastrigin(x[:25]-o[:25])
def C_Task1(x,o):
    x=x[:dimension_Ctask1]
    n = len(x)
    sum_term = np.sum((x-o)**2) / 4000
    prod_term = np.prod(np.cos((x-o) / np.sqrt(np.arange(1, n + 1))))
    return sum_term - prod_term + 1
def C_Task2(x,shift):
    x=x[:dimension_Ctask2]
    n = len(x)
    sum_sq_term = -0.2 * np.sqrt((1/n) * np.sum((x - shift) ** 2))
    cos_term = np.sum(np.cos(2 * np.pi * (x - shift)))
    return -20 * np.exp(sum_sq_term) - np.exp((1/n) * cos_term) + 20 + np.exp(1)

def C_Task3(x,o):
    x=x[:dimension_Ctask3]
    return np.sum((x-o)**2)

In [ ]:
def initialize_population(population_size, dimension_max):
    return np.random.uniform(0, 1, size=(population_size, dimension_max))

In [ ]:
def cost_Ctask1(population):
    result = []
    for x in population:
      result.append(C_Task1(x*100-50,shift_Ctask1));
    return np.array(result)
def cost_Ctask2(population):
    result = []
    for x in population:
      result.append(C_Task2(x*10-5,shift_Ctask2));
    return np.array(result)
def cost_Ctask3(population):
    result = []
    for x in population:
      result.append(C_Task3(x*100-50,shift_Ctask3));
    return np.array(result)
def cost_Etask(population):
    result = []
    for x in population:
      result.append(E_Task(x*100-50,shift_Etask));
    return np.array(result)

In [ ]:
def rank_Ctask1(population):
    return np.argsort(np.argsort(cost_Ctask1(population)))
def rank_Ctask2(population):
    return np.argsort(np.argsort(cost_Ctask2(population)))
def rank_Ctask3(population):
    return np.argsort(np.argsort(cost_Ctask3(population)))
def rank_Etask(population):
    return np.argsort(np.argsort(cost_Etask(population)))

In [ ]:
def evalution(population):
  rank_1=rank_Ctask1(population)
  rank_2=rank_Ctask2(population)
  rank_3=rank_Ctask3(population)
  rank_e=rank_Etask(population)
  skill_factor=[];
  for i in range(population_size):
        if (rank_1[i] <= rank_2[i] and rank_1[i]<=rank_3[i] and rank_1[i]<=rank_e[i]):
            skill_factor.append(1)
        elif(rank_2[i] <= rank_1[i] and rank_2[i]<=rank_3[i] and rank_2[i]<=rank_e[i]):
            skill_factor.append(2)
        elif(rank_3[i] <= rank_1[i] and rank_3[i]<=rank_2[i] and rank_3[i]<=rank_e[i]):
            skill_factor.append(3)
        elif(rank_e[i] <= rank_1[i] and rank_e[i]<=rank_2[i] and rank_e[i]<=rank_3[i]):
            skill_factor.append(4)
  return np.array(skill_factor)

In [ ]:
def crossover(parent1, parent2):
    cf=random.random()
    child1 =0.5*( parent1 * (1+cf) + parent2 * (1 - cf))
    child2 =0.5*( parent2 * (1+cf) + parent1 * (1 - cf))
    return child1, child2

In [ ]:
def mutation(individual, mutation_rate, mutation_scale):
    mutated_individual = individual.copy()
    for i in range(len(mutated_individual)):
        if (np.random.rand() < mutation_rate ):
            mutated_individual[i] += np.random.normal(scale=mutation_scale)
    return mutated_individual

In [ ]:
def get_m_k_Ctask1(population):
  Ctask1_values = cost_Ctask1(population)
  sorted_indices = np.argsort(Ctask1_values)
  num_elements = int(population_size * 0.4)
  selected_indices = sorted_indices[:num_elements]
  selected_points = population[selected_indices]
  return np.sum(selected_points, axis=0)/num_elements
def get_m_k_Ctask2(population):
  Ctask2_values = cost_Ctask2(population)
  sorted_indices = np.argsort(Ctask2_values)
  num_elements = int(population_size * 0.4)
  selected_indices = sorted_indices[:num_elements]
  selected_points = population[selected_indices]
  return np.sum(selected_points, axis=0)/num_elements
def get_m_k_Ctask3(population):
  Ctask3_values = cost_Ctask3(population)
  sorted_indices = np.argsort(Ctask3_values)
  num_elements = int(population_size * 0.4)
  selected_indices = sorted_indices[:num_elements]
  selected_points = population[selected_indices]
  return np.sum(selected_points, axis=0)/num_elements
def get_m_k_Etask(population):
  Etask_values = cost_Etask(population)
  sorted_indices = np.argsort(Etask_values)
  num_elements = int(population_size * 0.4)
  selected_indices = sorted_indices[:num_elements]
  selected_points = population[selected_indices]
  return np.sum(selected_points, axis=0)/num_elements

In [ ]:
def translate(gen, maxgen, population,frequency,threshold,skill_factor,d_k_Ctask1,d_k_Ctask2,d_k_Ctask3,d_k_Etask):
  NP=population.shape[0]
  population_=[]
  if(gen>threshold):
    if(gen%frequency==0):
      alpha=(gen/maxgen)*(gen/maxgen)
      d_k_Ctask1=sf*alpha*(cp-get_m_k_Ctask1(population))
      d_k_Ctask2=sf*alpha*(cp-get_m_k_Ctask2(population))
      d_k_Ctask3=sf*alpha*(cp-get_m_k_Ctask3(population))
      d_k_Etask=sf*alpha*(cp-get_m_k_Etask(population))
    for i in range(NP):
      if(skill_factor[i]==1):
        p_=population[i]+d_k_Ctask1
      elif(skill_factor[i]==2):
        p_=population[i]+d_k_Ctask2
      elif(skill_factor[i]==3):
        p_=population[i]+d_k_Ctask3
      elif(skill_factor[i]==4):
        p_=population[i]+d_k_Etask
      population_.append(p_)
  else:
    population_=population
  return population_,d_k_Ctask1,d_k_Ctask2,d_k_Ctask3,d_k_Etask

In [ ]:
def shuffle(parent1_,parent2_,population_,skill_factor, skill_factor_1,skill_factor_2):
  D1=len(parent1_)
  D2=len(parent2_)
  Dmax=max(D1,D2)
  l1=[i for i in range(0, Dmax)]
  l2=[i for i in range(0, Dmax)]
  if D1<D2:
    random.shuffle(l1)
    p_=population_[skill_factor.index(skill_factor_2)]
    for i in range(D1):
      p_[l1[i]]=parent1_[i]
    return p_,parent2_,l1,l2
  elif D1>D2:
    random.shuffle(l2)
    p_=population_[skill_factor.index(skill_factor_1)]
    for i in range(D2):
      p_[l2[i]]=parent2_[i]
    return parent1_,p_,l1,l2
  else:
    return parent1_,parent2_,l1,l2

In [ ]:
def re_transfer(o1__,o2__,l1,l2,len1,len2):
  if len1>len2:
    o2_=[None]*len2
    for i in range(len(l1)):
      o2_[l2[i]]=o2__[i]
    return o1__,o2_
  elif len1<len2:
    o1_=[None]*len1
    for i in range(len(l2)):
      o1_[l1[i]]=o1__[i]
    return o1_,o2__
  else:
    return o1__,o2__

In [ ]:
def assortative_mating(p1,p2, skill_factor1,skill_factor2):
    rmp=0.3
    rnd = np.random.rand(1)[0]
    if skill_factor1== skill_factor2 or rnd < rmp:
      o1,o2=crossover(p1,p2);
      random_skill_factor = np.random.rand(1)[0]
      if random_skill_factor < 0.5:
        return o1,o2,skill_factor1,skill_factor1
      else:
        return o1,o2,skill_factor2,skill_factor2
    else:
        o1= p1.copy()
        o2 = p2.copy()
        o1=mutation(o1,0.1,0.1);
        o2=mutation(o2,0.1,0.1);
        return o1,o2,skill_factor1,skill_factor2

In [ ]:
def update_scalar_fitness(population, s):
    Ctask1_pop = population[np.where(s == 1)[0]]
    rank1 = rank_Ctask1(Ctask1_pop)
    scalar_fitness_Ctask1 = [1/(i+1) for i in rank1]

    Ctask2_pop = population[np.where(s == 2)[0]]
    rank2 = rank_Ctask2(Ctask2_pop)
    scalar_fitness_Ctask2 = [1/(i+1) for i in rank2]

    Ctask3_pop = population[np.where(s == 3)[0]]
    rank3 = rank_Ctask3(Ctask3_pop)
    scalar_fitness_Ctask3 = [1/(i+1) for i in rank3]

    Etask_pop = population[np.where(s == 4)[0]]
    rank4 = rank_Etask(Etask_pop)
    scalar_fitness_Etask = [1/(i+1) for i in rank4]

    scalar_fitness = np.random.rand(population.shape[0])
    scalar_fitness[np.where(s == 1)[0]] = scalar_fitness_Ctask1
    scalar_fitness[np.where(s == 2)[0]] = scalar_fitness_Ctask2
    scalar_fitness[np.where(s == 3)[0]] = scalar_fitness_Ctask3
    scalar_fitness[np.where(s == 4)[0]] = scalar_fitness_Etask

    resize=len(scalar_fitness)-population_size
    filter_indices = np.argsort(scalar_fitness)[:resize]
    fittest_pop = np.delete(population, filter_indices, axis=0)
    fittest_skill_factor = np.delete(s, filter_indices)
    return fittest_pop, fittest_skill_factor

In [ ]:
def GMFEAs(NP):
  population_=[]
  d_k_Ctask1=0
  d_k_Ctask2=0;
  d_k_Ctask3=0;
  d_k_Etask=0;

  frequency=int(0.02*maxgen)
  threshold=int(0.1*maxgen)
  population=initialize_population(NP,dimension_max)
  skill_factor=evalution(population)
  for i in range(maxgen):
    population_,d_k_Ctask1,d_k_Ctask2,d_k_Ctask3,d_k_Etask=translate(i+1,maxgen,population,frequency,threshold,skill_factor,d_k_Ctask1,d_k_Ctask2,d_k_Ctask3,d_k_Etask)
    offspring=[]
    skill_factor_offspring=[]
    for j in range(int(NP/2)):
       random_indices = np.random.choice(np.arange(len(population_)), size=2, replace=False)
       p1_,p2_=population_[random_indices[0]],population_[random_indices[1]]
       p1__,p2__,l1,l2=shuffle(p1_,p2_,population_,skill_factor,skill_factor[random_indices[0]],skill_factor[random_indices[1]])
       o1__,o2__,skill_factor1,skill_factor2=assortative_mating(p1__,p2__,skill_factor[random_indices[0]],skill_factor[random_indices[1]])
       o1_,o2_=re_transfer(o1__,o2__,l1,l2,len(p1_),len(p2_))

       if(skill_factor1==1):
        o1=o1_-d_k_Ctask1
       elif(skill_factor1==2):
        o1=o1_-d_k_Ctask2
       elif(skill_factor1==3):
        o1=o1_-d_k_Ctask3
       else:
        o1=o1_-d_k_Etask

       if(skill_factor2==1):
        o2=o2_-d_k_Ctask1
       elif(skill_factor2==2):
        o2=o2_-d_k_Ctask2
       elif(skill_factor2==3):
        o2=o2_-d_k_Ctask3
       else:
        o2=o2_-d_k_Etask

       offspring.append(o1)
       offspring.append(o2)
       skill_factor_offspring.append(skill_factor1)
       skill_factor_offspring.append(skill_factor2)

    intermediate_pop = np.vstack((population, offspring))
    intermediate_skill_factor = np.concatenate((skill_factor,skill_factor_offspring))
    population, skill_factor = update_scalar_fitness(intermediate_pop, intermediate_skill_factor)

  Ctask1_pop = population[np.where(skill_factor == 1)[0]]
  Ctask2_pop = population[np.where(skill_factor == 2)[0]]
  Ctask3_pop = population[np.where(skill_factor == 3)[0]]
  Etask_pop = population[np.where(skill_factor == 4)[0]]

  print("Ctask1: ", cost_Ctask1(Ctask1_pop).min())
  print("Ctask2: ", cost_Ctask2(Ctask2_pop).min())
  print("Ctask3: ", cost_Ctask3(Ctask3_pop).min())
  print("Etask: ", cost_Etask(Etask_pop).min())

In [ ]:
GMFEAs(population_size)

Ctask1:  0.050635867979107685
Ctask2:  0.016301274336541827
Ctask3:  8.439299373385326
Etask:  278.5533177809508
